# 期货技术指标分析系统 - 完整教程

本教程将教会你如何使用 Streamlit 构建一个完整的期货技术指标分析系统。

## 学习目标

1. 理解 Streamlit 框架的基本使用
2. 掌握 AKShare 数据获取方法
3. 学会计算 RSI 和 KDJ 技术指标
4. 使用 Matplotlib 绘制多子图
5. 实现交互式数据可视化应用

## 系统功能

- ✅ 支持 IF/IM/IC/IH 四个标的切换
- ✅ 支持分钟级和日级数据切换
- ✅ 主图：收盘价折线图
- ✅ 副图：RSI 和 KDJ 技术指标
- ✅ 日期范围筛选
- ✅ 完整的容错处理



## 第一部分：环境准备

首先需要安装必要的库：

```bash
pip install streamlit akshare pandas numpy matplotlib
```

让我们先导入所有需要的库：


In [ ]:
# 导入必要的库
import streamlit as st
import akshare as ak
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ 所有库导入成功！")


## 第二部分：数据获取模块

### 2.1 标的代码映射

期货连续合约的代码规则：品种代码 + 0
- IF0: 沪深300主力连续
- IM0: 中证1000主力连续  
- IC0: 中证500主力连续
- IH0: 上证50主力连续


In [ ]:
# 标的代码映射字典
SYMBOL_MAP = {
    "IF": "IF0",  # 沪深300主力连续
    "IM": "IM0",  # 中证1000主力连续
    "IC": "IC0",  # 中证500主力连续
    "IH": "IH0"   # 上证50主力连续
}

print("标的代码映射：")
for key, value in SYMBOL_MAP.items():
    print(f"  {key} -> {value}")


### 2.2 获取日级数据

使用 `ak.futures_zh_daily_sina()` 接口获取日级K线数据。

**接口说明：**
- 输入参数：`symbol` (合约代码，如 "IF0")
- 返回数据：包含 date, open, high, low, close, volume, hold, settle 等列


In [ ]:
def get_daily_data(symbol: str) -> pd.DataFrame:
    """
    获取日级期货数据
    
    参数:
        symbol: 合约代码，如 "IF0"
    
    返回:
        DataFrame: 包含日期、开高低收等数据的DataFrame
    """
    try:
        # 调用 AKShare 接口获取数据
        df = ak.futures_zh_daily_sina(symbol=symbol)
        
        # 检查数据是否为空
        if df is None or df.empty:
            print(f"⚠️ 获取 {symbol} 日级数据失败：数据为空")
            return None
        
        # 数据清洗和格式化
        # 1. 将日期列转换为 datetime 类型并设为索引
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
        
        # 2. 确保数值列为数字类型
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # 3. 按日期排序
        df = df.sort_index()
        
        return df
        
    except Exception as e:
        print(f"❌ 获取日级数据时出错: {str(e)}")
        return None

# 测试获取日级数据
print("正在获取 IF0 的日级数据...")
df_daily = get_daily_data("IF0")
if df_daily is not None:
    print(f"✅ 成功获取 {len(df_daily)} 条日级数据")
    print("\n数据预览：")
    print(df_daily.head())
    print(f"\n数据列：{df_daily.columns.tolist()}")
else:
    print("❌ 数据获取失败")


### 2.3 获取分钟级数据

使用 `ak.futures_zh_minute_sina()` 接口获取分钟级K线数据。

**接口说明：**
- 输入参数：
  - `symbol`: 合约代码，如 "IF0"
  - `period`: 周期，"1" 表示1分钟
- 返回数据：包含 datetime, open, high, low, close, volume 等列


In [ ]:
def get_minute_data(symbol: str, period: str = "1") -> pd.DataFrame:
    """
    获取分钟级期货数据
    
    参数:
        symbol: 合约代码，如 "IF0"
        period: 周期，"1" 表示1分钟
    
    返回:
        DataFrame: 包含时间、开高低收等数据的DataFrame
    """
    try:
        # 调用 AKShare 接口获取分钟级数据
        df = ak.futures_zh_minute_sina(symbol=symbol, period=period)
        
        # 检查数据是否为空
        if df is None or df.empty:
            print(f"⚠️ 获取 {symbol} 分钟级数据失败：数据为空")
            return None
        
        # 数据清洗和格式化
        # 1. 将日期时间列转换为 datetime 类型并设为索引
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
            df = df.set_index('datetime')
        
        # 2. 确保数值列为数字类型
        numeric_columns = ['open', 'high', 'low', 'close', 'volume']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # 3. 按时间排序
        df = df.sort_index()
        
        return df
        
    except Exception as e:
        print(f"❌ 获取分钟级数据时出错: {str(e)}")
        return None

# 测试获取分钟级数据（注意：分钟级数据量很大，可能较慢）
print("正在获取 IF0 的分钟级数据（1分钟K线）...")
print("⚠️ 注意：分钟级数据量较大，可能需要一些时间...")
df_minute = get_minute_data("IF0")
if df_minute is not None:
    print(f"✅ 成功获取 {len(df_minute)} 条分钟级数据")
    print("\n数据预览（最新5条）：")
    print(df_minute.tail())
    print(f"\n数据时间范围：{df_minute.index.min()} 至 {df_minute.index.max()}")
else:
    print("❌ 数据获取失败")


## 第三部分：技术指标计算

### 3.1 RSI 指标计算

**RSI (Relative Strength Index) 相对强弱指标**

计算公式：
1. 计算价格变化：`delta = close.diff()`
2. 计算上涨幅度平均值：`gain = delta.where(delta > 0, 0).rolling(14).mean()`
3. 计算下跌幅度平均值：`loss = (-delta.where(delta < 0, 0)).rolling(14).mean()`
4. 计算相对强度：`RS = gain / loss`
5. 计算RSI：`RSI = 100 - (100 / (1 + RS))`

**RSI 解读：**
- RSI > 70：超买区域，可能下跌
- RSI < 30：超卖区域，可能上涨
- RSI 在 30-70 之间：正常波动区间


In [ ]:
def calculate_rsi(prices: pd.Series, period: int = 14) -> pd.Series:
    """
    计算RSI指标
    
    参数:
        prices: 价格序列（通常是收盘价）
        period: 计算周期，默认14
    
    返回:
        Series: RSI值序列
    """
    # 1. 计算价格变化
    delta = prices.diff()
    
    # 2. 分离上涨和下跌
    # where(condition, other): 满足条件保留原值，否则设为other
    gain = delta.where(delta > 0, 0)  # 只保留上涨的部分
    loss = -delta.where(delta < 0, 0)  # 只保留下跌的部分（转为正数）
    
    # 3. 计算滚动平均（滑动窗口）
    # rolling(window=period): 创建滑动窗口
    # mean(): 计算窗口内的平均值
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # 4. 计算相对强度 RS
    # 1e-10 是一个很小的数，用于避免除零错误
    rs = avg_gain / (avg_loss + 1e-10)
    
    # 5. 计算RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# 测试RSI计算
if df_daily is not None:
    df_daily['RSI'] = calculate_rsi(df_daily['close'])
    print("✅ RSI计算完成")
    print("\nRSI数据预览（最新10条）：")
    print(df_daily[['close', 'RSI']].tail(10))
    
    # 显示RSI统计信息
    print(f"\nRSI统计信息：")
    print(f"  最大值: {df_daily['RSI'].max():.2f}")
    print(f"  最小值: {df_daily['RSI'].min():.2f}")
    print(f"  平均值: {df_daily['RSI'].mean():.2f}")
    print(f"  当前值: {df_daily['RSI'].iloc[-1]:.2f}")


### 3.2 KDJ 指标计算

**KDJ 指标（随机指标）**

KDJ 是基于最高价、最低价、收盘价计算的动量指标。

计算公式：
1. 计算 RSV（未成熟随机值）：
   - `low_list = low.rolling(9).min()`  # 9日最低价
   - `high_list = high.rolling(9).max()`  # 9日最高价
   - `RSV = (close - low_list) / (high_list - low_list) * 100`

2. 计算 K 值（快速随机指标）：
   - `K = RSV.ewm(span=3).mean()`  # 指数移动平均

3. 计算 D 值（慢速随机指标）：
   - `D = K.ewm(span=3).mean()`  # K值的指数移动平均

4. 计算 J 值：
   - `J = 3 * K - 2 * D`

**KDJ 解读：**
- K > 80 或 D > 80：超买区域
- K < 20 或 D < 20：超卖区域
- J > 100：极度超买
- J < 0：极度超卖


In [ ]:
def calculate_kdj(df: pd.DataFrame, n: int = 9, m1: int = 3, m2: int = 3):
    """
    计算KDJ指标
    
    参数:
        df: 包含 high, low, close 列的DataFrame
        n: 计算周期，默认9
        m1: K值平滑周期，默认3
        m2: D值平滑周期，默认3
    
    返回:
        tuple: (K, D, J) 三个Series
    """
    # 1. 计算n日内的最低价和最高价
    low_list = df['low'].rolling(window=n).min()   # 滚动窗口最小值
    high_list = df['high'].rolling(window=n).max()  # 滚动窗口最大值
    
    # 2. 计算RSV（未成熟随机值）
    # RSV = (收盘价 - 最低价) / (最高价 - 最低价) * 100
    rsv = (df['close'] - low_list) / (high_list - low_list + 1e-10) * 100
    
    # 3. 计算K值（指数移动平均）
    # ewm(span=m1): 指数加权移动平均，span是平滑系数
    k = rsv.ewm(span=m1).mean()
    
    # 4. 计算D值（K值的指数移动平均）
    d = k.ewm(span=m2).mean()
    
    # 5. 计算J值
    j = 3 * k - 2 * d
    
    return k, d, j

# 测试KDJ计算
if df_daily is not None:
    df_daily['K'], df_daily['D'], df_daily['J'] = calculate_kdj(df_daily)
    print("✅ KDJ计算完成")
    print("\nKDJ数据预览（最新10条）：")
    print(df_daily[['close', 'K', 'D', 'J']].tail(10))
    
    # 显示KDJ统计信息
    print(f"\nKDJ统计信息：")
    print(f"K值范围: [{df_daily['K'].min():.2f}, {df_daily['K'].max():.2f}]")
    print(f"D值范围: [{df_daily['D'].min():.2f}, {df_daily['D'].max():.2f}]")
    print(f"J值范围: [{df_daily['J'].min():.2f}, {df_daily['J'].max():.2f}]")


## 第四部分：数据筛选

根据用户选择的日期范围筛选数据。


In [ ]:
def filter_data_by_date(df: pd.DataFrame, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    """
    根据日期范围筛选数据
    
    参数:
        df: 原始数据DataFrame（索引必须是日期时间）
        start_date: 开始日期
        end_date: 结束日期
    
    返回:
        DataFrame: 筛选后的数据
    """
    # 创建布尔掩码：筛选在日期范围内的数据
    mask = (df.index >= start_date) & (df.index <= end_date)
    return df[mask]

# 测试日期筛选
if df_daily is not None:
    # 选择最近30天的数据
    end_date = df_daily.index.max()
    start_date = end_date - timedelta(days=30)
    
    df_filtered = filter_data_by_date(df_daily, start_date, end_date)
    print(f"✅ 日期筛选完成")
    print(f"原始数据: {len(df_daily)} 条")
    print(f"筛选后数据: {len(df_filtered)} 条")
    print(f"日期范围: {start_date.date()} 至 {end_date.date()}")


## 第五部分：图表绘制

使用 Matplotlib 绘制多子图：
1. 主图：收盘价折线图
2. 副图1：RSI 指标（带超买超卖线）
3. 副图2：KDJ 指标（K、D、J三条线）


In [ ]:
def plot_charts(df: pd.DataFrame, symbol: str, data_type: str):
    """
    绘制主图和副图
    
    参数:
        df: 数据DataFrame（必须包含 close, RSI, K, D, J 列）
        symbol: 标的代码（用于标题）
        data_type: 数据类型（"分钟级" 或 "日级"）
    """
    if df is None or df.empty:
        print("⚠️ 数据为空，无法绘制图表")
        return
    
    # 创建3个子图：主图 + 2个副图
    # figsize: 图表大小 (宽度, 高度)
    # sharex: 共享x轴
    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    
    # ========== 主图：收盘价折线图 ==========
    axes[0].plot(df.index, df['close'], label='收盘价', color='blue', linewidth=1.5)
    axes[0].set_title(f'{symbol} {data_type}收盘价走势图', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('价格', fontsize=12)
    axes[0].legend(loc='best')
    axes[0].grid(True, alpha=0.3)  # 显示网格，alpha控制透明度
    
    # ========== 副图1：RSI指标 ==========
    axes[1].plot(df.index, df['RSI'], label='RSI(14)', color='orange', linewidth=1.5)
    # 添加超买超卖参考线
    axes[1].axhline(y=70, color='red', linestyle='--', alpha=0.7, label='超买线(70)')
    axes[1].axhline(y=30, color='green', linestyle='--', alpha=0.7, label='超卖线(30)')
    # 填充30-70之间的区域
    axes[1].fill_between(df.index, 30, 70, alpha=0.1, color='gray')
    axes[1].set_ylabel('RSI', fontsize=12)
    axes[1].set_ylim(0, 100)  # RSI范围是0-100
    axes[1].legend(loc='best')
    axes[1].grid(True, alpha=0.3)
    
    # ========== 副图2：KDJ指标 ==========
    axes[2].plot(df.index, df['K'], label='K', color='blue', linewidth=1.5)
    axes[2].plot(df.index, df['D'], label='D', color='red', linewidth=1.5)
    axes[2].plot(df.index, df['J'], label='J', color='purple', linewidth=1.5)
    # 添加超买超卖参考线
    axes[2].axhline(y=80, color='red', linestyle='--', alpha=0.3)
    axes[2].axhline(y=20, color='green', linestyle='--', alpha=0.3)
    axes[2].set_ylabel('KDJ', fontsize=12)
    axes[2].set_xlabel('时间', fontsize=12)
    axes[2].legend(loc='best')
    axes[2].grid(True, alpha=0.3)
    
    # 格式化x轴日期显示
    if data_type == "日级":
        axes[2].xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
    else:
        axes[2].xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%m-%d %H:%M'))
    
    # 旋转x轴标签，避免重叠
    plt.xticks(rotation=45)
    
    # 调整子图间距
    plt.tight_layout()
    
    # 显示图表
    plt.show()

# 测试绘制图表
if df_daily is not None and 'RSI' in df_daily.columns:
    print("正在绘制图表...")
    plot_charts(df_daily.tail(60), "IF", "日级")  # 只绘制最近60条数据
else:
    print("⚠️ 请先计算技术指标")


## 第六部分：Streamlit 应用集成

### 6.1 Streamlit 基础组件

**常用组件说明：**
- `st.title()`: 页面标题
- `st.sidebar`: 侧边栏
- `st.selectbox()`: 下拉选择框
- `st.radio()`: 单选按钮
- `st.date_input()`: 日期选择器
- `st.button()`: 按钮
- `st.spinner()`: 加载提示
- `st.metric()`: 指标卡片
- `st.pyplot()`: 显示matplotlib图表
- `st.dataframe()`: 显示DataFrame表格


### 6.2 完整的 Streamlit 应用代码

完整的应用代码已保存在 `streamlit.py` 文件中。

**运行方式：**
```bash
streamlit run streamlit.py
```

**应用结构：**
1. **侧边栏**：参数设置
   - 标的选择（IF/IM/IC/IH）
   - 数据类型选择（日级/分钟级）
   - 日期选择器
   - 刷新按钮

2. **主内容区**：
   - 数据统计卡片
   - 技术指标图表
   - 原始数据表格

3. **容错处理**：
   - 数据获取失败提示
   - 日期范围校验
   - 空数据提示


## 第七部分：关键知识点总结

### 7.1 数据获取

**AKShare 接口：**
- 日级数据：`ak.futures_zh_daily_sina(symbol="IF0")`
- 分钟级数据：`ak.futures_zh_minute_sina(symbol="IF0", period="1")`

**注意事项：**
- 连续合约代码 = 品种代码 + "0"（如 IF0）
- 分钟级数据量很大，建议限制日期范围
- 需要处理网络异常和数据为空的情况

### 7.2 技术指标计算

**RSI 计算要点：**
- 使用 `rolling()` 创建滑动窗口
- 使用 `where()` 条件筛选
- 避免除零错误（+ 1e-10）

**KDJ 计算要点：**
- 使用 `rolling().min()` 和 `rolling().max()` 获取极值
- 使用 `ewm()` 计算指数移动平均
- J = 3K - 2D 是经验公式

### 7.3 图表绘制

**Matplotlib 多子图：**
- `plt.subplots(3, 1)`: 创建3行1列的子图
- `sharex=True`: 共享x轴
- `tight_layout()`: 自动调整布局

**日期格式化：**
- 日级：`DateFormatter('%Y-%m-%d')`
- 分钟级：`DateFormatter('%m-%d %H:%M')`

### 7.4 Streamlit 交互

**状态管理：**
- Streamlit 每次交互都会重新运行整个脚本
- 使用 `st.rerun()` 手动刷新
- 使用 `st.session_state` 保存状态（高级用法）

**性能优化：**
- 使用 `@st.cache_data` 缓存数据（可选）
- 限制数据量，避免加载过大数据集


## 第八部分：常见问题与解决方案

### Q1: 数据获取失败怎么办？

**可能原因：**
- 网络连接问题
- 合约代码错误
- AKShare 接口更新

**解决方案：**
- 检查网络连接
- 确认合约代码正确（IF0, IM0, IC0, IH0）
- 更新 AKShare：`pip install --upgrade akshare`

### Q2: 分钟级数据加载很慢？

**原因：**
- 分钟级数据量非常大（一天有240+条数据）

**解决方案：**
- 限制日期范围（只选择单日或几天）
- 使用 `@st.cache_data` 缓存数据
- 考虑使用日级数据

### Q3: RSI/KDJ 计算出现 NaN？

**原因：**
- 前N条数据不足以计算指标（如RSI需要14条数据）

**解决方案：**
- 这是正常现象，前14条数据的RSI会是NaN
- 使用 `dropna()` 删除NaN值，或直接忽略

### Q4: 图表显示不正常？

**可能原因：**
- 数据索引不是日期时间类型
- 数据为空或格式错误

**解决方案：**
- 确保使用 `pd.to_datetime()` 转换日期
- 检查数据是否为空
- 确认数据列名正确（close, high, low等）

### Q5: Streamlit 应用如何部署？

**部署方式：**
1. **本地运行**：`streamlit run streamlit.py`
2. **Streamlit Cloud**：上传到 GitHub，在 Streamlit Cloud 部署
3. **Docker**：使用 Docker 容器化部署
4. **服务器部署**：使用 nginx + systemd 部署

**推荐：Streamlit Cloud（免费）**
- 访问：https://streamlit.io/cloud
- 连接 GitHub 仓库
- 自动部署和更新


## 第九部分：扩展练习

### 练习1：添加更多技术指标

尝试添加以下指标：
- MACD（指数平滑移动平均线）
- BOLL（布林带）
- MA（移动平均线）

### 练习2：添加交易信号

基于 RSI 和 KDJ 生成买卖信号：
- RSI < 30 且 K < 20：买入信号
- RSI > 70 且 K > 80：卖出信号

### 练习3：数据导出功能

添加数据导出功能：
- 导出为 CSV 文件
- 导出为 Excel 文件
- 导出图表为图片

### 练习4：多标的对比

实现多标的对比功能：
- 同时显示多个标的的价格走势
- 对比不同标的的技术指标

---

## 总结

通过本教程，你已经学会了：
1. ✅ 使用 AKShare 获取期货数据
2. ✅ 计算 RSI 和 KDJ 技术指标
3. ✅ 使用 Matplotlib 绘制多子图
4. ✅ 使用 Streamlit 构建交互式应用
5. ✅ 处理异常和错误情况

**下一步：**
- 运行 `streamlit run streamlit.py` 查看完整应用
- 根据需求自定义功能
- 尝试添加更多技术指标

**祝你学习愉快！** 🎉
